<a href="https://colab.research.google.com/github/chinmaydashp/LeakDetection/blob/main/Main_Model_with_Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://news.illinois.edu/files/6367/543635/116641.jpg" alt="University of Illinois" width="250"/>

# Shell Project - Leak Detection #

* Richard Sowers, <r-sowers@illinois.edu>, <https://publish.illinois.edu/r-sowers/>
* Ramavarapu S Sreenivas, <rsree@illinois.edu>, <http://rsree.ise.illinois.edu/>
* Ayse Dogan, <adogan2@illinois.edu>
* Chinmay Daspute, <chinmay5@illinois.edu>
* Tiffany Yuan, 




Copyright 2020 University of Illinois Board of Trustees. All Rights Reserved.

Overleaf is at https://www.overleaf.com/project/61872fcbb97f89c84ec25792

# imports 

In [ ]:
import os
import sys
import datetime
import torch
import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

# configuration variables

In [ ]:
def getfile(location_pair,**kwargs): #tries to get local version and then defaults to google drive version
    (loc,gdrive)=location_pair
    try:
        out=pd.read_csv(loc,**kwargs)
    except FileNotFoundError:
        print("local file not found; accessing Google Drive")
        loc = 'https://drive.google.com/uc?export=download&id='+gdrive.split('/')[-2]
        out=pd.read_csv(loc,**kwargs)
    return out

### configuration variables###

In [ ]:
#training data
# ticker_A="data"
# filedata_A=(ticker_A+".csv","https://drive.google.com/file/d/1cf55NGvanAvsCCZL8G6d8s-UHk5kK3Jw/view?usp=sharing")


# #the main file for model
ticker_B="NG Profile"
filedata_B=(ticker_B+".csv","https://drive.google.com/file/d/1OYgeQkwpxVsCth_AXEzWRCfLyZmgHwUN/view?usp=sharing")



# load data #

In [ ]:
df = getfile(filedata_B)
NG_Profile = df.copy()

local file not found; accessing Google Drive


In [ ]:
def initialize_df():
    # Molar flow - kmole / hour
    df_moles = pd.DataFrame(columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'N2', 'CO2'], 
                            index = ['FEED_SC', 'COND', 'FEED_GSU', 'CO2', 'FEED_NGL'])    
    # Molar flow - kg / hour
    df_mass = pd.DataFrame(columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'N2', 'CO2'], 
                           index = ['FEED_SC', 'COND', 'FEED_GSU', 'CO2', 'FEED_NGL'])
    
    df_output = pd.DataFrame(columns = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'N2', 'CO2'], 
                            index = ['FEED_SC', 'COND', 'FEED_GSU', 'CO2', 'FEED_NGL'])
    
    return df_moles, df_mass, df_output


In [ ]:
def mol_wt_calculation(x_feedgas,mol_wt): 
    x_feedgas = np.array([x_feedgas[0], x_feedgas[1], x_feedgas[2], # Array of gas compositions (%) in the feedgas 
                          x_feedgas[3] + x_feedgas[4], 
                          x_feedgas[5] + x_feedgas[6], 
                          x_feedgas[7], x_feedgas[8], x_feedgas[9]]) # Array of Molecular weight of individual gas components
    mol_wt = np.array([mol_wt[0], mol_wt[1], mol_wt[2], mol_wt[3], mol_wt[5], mol_wt[7], mol_wt[8], mol_wt[9]])
    mol_wt_feedgas = np.dot(x_feedgas,mol_wt)/100   # mol_wt_feedgas is a scalar variable for the molar weight of the input feedgas.
    
    return(x_feedgas, mol_wt, mol_wt_feedgas)

To convert from the df_moles to df_mass dataframe, we are just multiplying each component of the df_moles dataframe by their respective molecular weights. 

In [ ]:
def unit_inlet(df_moles, df_mass, m_feedgas, mol_wt_feedgas, x_feedgas, mol_wt):
    df_moles.loc['FEED_SC'] = 0
    df_moles.loc['FEED_SC'] = m_feedgas * 1000 / mol_wt_feedgas * x_feedgas /100  # Array of component-wise Molar feedgas flow through the Feed_SC pipeline in kmoles/hr
    df_mass.loc['FEED_SC'] = df_moles.loc['FEED_SC'] * mol_wt                     # Array of component-wise Molar feedgas flow through the Feed_SC pipeline in kg/hr
    
    return df_moles, df_mass


In [ ]:
def unit_slug_catcher(df_moles, df_mass, mol_wt, leak_location, leak_rate):     # Slug-Catcher unit removes the C5 and C6 components from the feedgas.
    df_moles.loc['COND'] = 0                                                    # Cond is the C5 and C6 condensed liquids that are removed from the feedgas.
    df_moles.loc['COND'][['C5','C6']] = df_moles.loc['FEED_SC'][['C5','C6']]    # Molar flow of C5 and C6 components are added to 'COND', with the rest components being 0.
    df_moles.loc['FEED_GSU'] = 0                                                # Feed_GSU takes the rest of the components to next unit.
    df_moles.loc['FEED_GSU'][['C1','C2','C3','C4','N2','CO2']] = df_moles.loc['FEED_SC'][['C1','C2','C3','C4','N2','CO2']] 
    df_mass.loc['FEED_SC'] = df_moles.loc['FEED_SC'] * mol_wt
    df_mass.loc['COND'] = df_moles.loc['COND'] * mol_wt
    df_mass.loc['FEED_GSU'] = df_moles.loc['FEED_GSU'] * mol_wt
    
    if "COND" in leak_location:
        df_moles.loc['COND'] = (100-leak_rate)/100 * df_moles.loc['COND']             # If leak is taking place in 'COND' pipeline, reducing output by defined leak rate.
        df_mass.loc['COND'] = (100-leak_rate)/100 * df_mass.loc['COND']
        df_moles.loc['FEED_GSU'] = (100-leak_rate)/100 * df_moles.loc['FEED_GSU']               # If leak is taking place in 'FEED_GSU' pipeline, reducing output by defined leak rate.
        df_mass.loc['FEED_GSU'] = (100-leak_rate)/100 * df_mass.loc['FEED_GSU']    

    return df_moles, df_mass


In [ ]:
def unit_gsu(df_moles, df_mass, mol_wt, leak_location, leak_rate):    # Gas Sweetening Unit removes CO2 from the feedgas.
    df_moles.loc['CO2'] = 0                                           
    df_moles.loc['CO2']['CO2'] = df_moles.loc['FEED_GSU']['CO2']      # Molar flow of CO2 removed from the feedgas
    df_moles.loc['FEED_NGL'] = 0
    df_moles.loc['FEED_NGL'][['C1','C2','C3','C4','N2']] = df_moles.loc['FEED_GSU'][['C1','C2','C3','C4','N2']] # Molar flow of the rest of the components carried to NGL unit
    df_mass.loc['CO2'] = df_moles.loc['CO2'] * mol_wt
    df_mass.loc['FEED_NGL'] = df_moles.loc['FEED_NGL'] * mol_wt

    if "CO2" in leak_location:
        df_moles.loc['CO2'] = (100-leak_rate)/100 * df_moles.loc['CO2']     # If leak is taking place in 'CO2' pipeline, reducing output by defined leak rate.
        df_mass.loc['CO2'] = (100-leak_rate)/100 * df_mass.loc['CO2']
        df_moles.loc['FEED_NGL'] = (100-leak_rate)/100 * df_moles.loc['FEED_NGL'] # If leak is taking place in 'FEED_NGL' pipeline, reducing output by defined leak rate.
        df_mass.loc['FEED_NGL'] = (100-leak_rate)/100 * df_mass.loc['FEED_NGL']

    return df_moles, df_mass


In [ ]:
def output_composition(df_mass, sensor_noise, df_output):

    df_mass_noise = df_mass.copy()
    
    df_output.loc['FEED_SC'] = df_mass_noise.loc['FEED_SC']*(1 + sensor_noise[0]/100)
    df_output.loc['COND'] = df_mass_noise.loc['COND']*(1 + sensor_noise[1]/100)
    df_output.loc['FEED_GSU'] = df_mass_noise.loc['FEED_GSU']*(1 + sensor_noise[2]/100)
    df_output.loc['CO2'] = df_mass_noise.loc['CO2']*(1 + sensor_noise[3]/100)
    df_output.loc['FEED_NGL'] = df_mass_noise.loc['FEED_NGL']*(1 + sensor_noise[4]/100)

    return df_output


In [ ]:
def mass_composition_energy(m_feedgas, x_feedgas, p_feedgas, t_feedgas, t_water, t_air, t_process, 
                            leak_location, leak_rate, mol_wt,sensor_noise):
     
    # Initialize
    df_moles, df_mass, df_output = initialize_df()

    # Molecular weight calculation
    x_feedgas, mol_wt, mol_wt_feedgas = mol_wt_calculation(x_feedgas,mol_wt)
    
    # Unit: Terminal Inlet
    df_moles, df_mass = unit_inlet(df_moles, df_mass, m_feedgas, mol_wt_feedgas, x_feedgas, mol_wt)

    # Unit: Slug Catcher
    df_moles, df_mass = unit_slug_catcher(df_moles, df_mass, mol_wt, leak_location, leak_rate)

    # Unit: Gas Treatment Section
    df_moles, df_mass = unit_gsu(df_moles, df_mass, mol_wt, leak_location, leak_rate)
            
    # Add noise to composition measurement and output to dataframe
    df_output = output_composition(df_mass, sensor_noise, df_output)

    return df_moles, df_mass, df_output

In [ ]:
def model_pd(leak_percentage,leak_location=[],N=1000,start_date='2018-06-10 00:00:00',):

    ## Input Parameters

    #### DataSet parameters
    MAX_NOISE = 2     # for input to gause(0,MAX_NOISE)

    #df=getfile(filedata_B)
    df = NG_Profile.copy()

    def perdelta(start, dur, delta):
        random.seed(0)
        i1=random.randint(0,len(df)-dur-1)
        count=0
        ts_ind=[]
        curr = start
        ts_ind.append(datetime.datetime.strftime(curr,'%Y-%m-%d %H:%M:%S'))
        while count < dur-1:
            curr += delta
            ts_ind.append(datetime.datetime.strftime(curr,'%Y-%m-%d %H:%M:%S'))
            count=count+1
        return df.iloc[i1:i1+dur]['Temp'].values,ts_ind,df.iloc[i1:i1+dur]['NG'].values
    temp,ts,feed=perdelta(datetime.datetime.strptime(start_date,'%Y-%m-%d %H:%M:%S'),N, datetime.timedelta(hours=1))
    feed_df=pd.DataFrame({'feed':feed,'temp':temp},index=ts)

    #### Transient Parameters

    process_variation = 0.01*0
    # process_noise = 0.01*np.random.normal(0,MAX_NOISE)
    process_noise = 0
    # sensor_noise = np.random.normal(0,MAX_NOISE)
    # noise = np.random.normal(0,MAX_NOISE)
    # sensor_noise = np.ones(5)*noise
    sensor_noise = []
    
    for i in range(5):
      noise = np.random.normal(5,0.6)      # Range of noise% is (0,x)
      sensor_noise = sensor_noise + [noise]
      # print('Sensor Noise',sensor_noise)
      i += 1 

    #### Feedgas properties

    # Feedgas - Mass flow rate in ton / h
    m_feedgas = 816

    # Feedgas - Composition in mole percentage
    # C1, C2, C3, nC4, iC4, nC5, iC5, C6+, N2, CO2, 
    x_feedgas = np.array([80.5, 5.1, 2.6, 0.8, 0.5, 0.5, 0.5, 1.8, 2.5, 5.200])

    # Feedgas - Conditions
    p_feedgas = 70   # bar
    t_feedgas = 1    # degC

    #### Ambient conditions

    # Ambient Conditions
    t_water = 7      # degC
    t_air = 7        # degC
    t_process = 17   # degC

    #### Feedgas properties - Transient Profile

    # Profile for feedgas flow rate
    m_feedgas_profile=feed_df['feed'].values # Retrieved from NG Profile.xlsx
    # m_feedgas_profile = m_feedgas_profile + m_feedgas_profile*(random.rand(N)-0.5)*process_noise

    # Profile for feedgas temperature
    t_feedgas_profile = t_feedgas*(1+process_variation*np.sin(np.pi+0.25*np.arange(N)))
    t_feedgas_profile = t_feedgas_profile + t_feedgas_profile*(random.rand(N)-0.5)*process_noise

    # Profile for feedgas pressure
    p_feedgas_profile = p_feedgas*np.ones(N)
    p_feedgas_profile = p_feedgas_profile + p_feedgas_profile*(random.rand(N)-0.5)*process_noise

    #### Feedgas Composition - Transient profile

    # Profile for feedgas C1 mole %
    xC1_feedgas_profile = x_feedgas[0]*(1+process_variation*np.sin(0.5*np.arange(N)))
    xC1_feedgas_profile = xC1_feedgas_profile + xC1_feedgas_profile*(random.rand(N)-0.5)*process_noise

    x_feedgas_profile = np.zeros((N,len(x_feedgas)))
    x_feedgas_profile[:,:] = x_feedgas
    x_feedgas_profile[:,0] = xC1_feedgas_profile[:]
    for i in range(N):
        x_feedgas_profile[i,:] = 100*x_feedgas_profile[i,:] /  x_feedgas_profile[i,:].sum() 

    #### Ambient Conditions - Transient profile

    # Profile for ambient air temperature
    t_air_profile=feed_df['temp'].values # Retrieved from NG Profile.xlsx

    # Profile for ambient water temperature
    t_water_profile = t_water*(1+process_variation*np.sin(np.pi+0.25*np.arange(N)))
    t_water_profile = t_water_profile + t_water_profile*(random.rand(N)-0.5)*process_noise

    # Profile for process temperature
    t_process_profile = t_process*np.ones(N)

    #### Leak Location

    # Input Leak location. Choose between NGL_C2, LNG, Feed_NGL, CO2, LPG. Options provided below.
    location = ["FEED_SC", "COND", "FEED_GSU", "CO2", "FEED_NGL"]

    #### Leak Profile

    # Profile for feedgas flow rate
    leak_profile = leak_percentage * m_feedgas_profile / np.mean(m_feedgas_profile)


    ## Initialization

    # Molecular weight of feedgas componentss
    mol_wt = np.array([16.04, 30.07, 44.1, 58.12, 58.12, 72.15, 72.15, 86.18, 28.01, 44.01])

    streams_label = []

    streams_names = ['FEED_SC', 'COND', 'FEED_GSU', 'CO2', 'FEED_NGL']
    streams_properties = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'N2', 'CO2']

    for i in streams_names:
        for j in streams_properties:
            label = i+'_'+j
            streams_label = streams_label +  [label]

    # Initialize Output Data Frame
    df = pd.DataFrame(columns = streams_label, index = np.arange(N))
    df_m = pd.DataFrame()
    ## Mass, Species, Energy Calculation
    counter = 0
    for counter in tqdm(range(N), position = 0, leave = True):
        df_moles, df_mass, df_output = mass_composition_energy(m_feedgas_profile[counter], x_feedgas_profile[counter,:], p_feedgas_profile[counter], t_feedgas_profile[counter], t_water_profile[counter], t_air_profile[counter], t_process_profile[counter],leak_location, leak_profile[counter], mol_wt,sensor_noise)
        # print(df_mass.loc['FEED_SC'])
        df_m = df_m.append(df_mass.loc['FEED_SC'])
        for i in df_output.index:
            for j in df_output.columns:
                label = i+'_'+j
                df.iloc[counter][label]=df_output.loc[i][j]
 
    return df,df_mass, df_m

# **LNG Plant Model (Generate Data)**

In [ ]:
def output_data(leak_rate,leak_location,n_timestamps):

  streams_names = ['FEED_SC', 'COND', 'FEED_GSU', 'CO2', 'FEED_NGL']
  streams_properties = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'N2', 'CO2']
  x = len(streams_names)*len(streams_properties)

  y = np.zeros([n_timestamps,x])
  index = 0
  streams_label = []

  # print(leak_location)
  # CO2 = "CO2"
  # COND = "COND"

  if leak_location == "COND":
    # print('COND')
    index = 1
  elif leak_location == "CO2":
    # print('CO2')
    index = 3

  # print(index)

  for i in range(n_timestamps):
    if index > 0:
      y[i,index*8:index*8+16] = leak_rate

  for i in streams_names:
      for j in streams_properties:
          label = i+'_'+j+'_y'
          streams_label = streams_label +  [label]

  # Initialize Output Data Frame
  df = pd.DataFrame(y,columns = streams_label)

  return df



To generate data, run the following cell. It has the leak locations defined.

In [ ]:
iter = 20             # Number of iterations, different leak location 
n_timestamps = 20     # Number of timesteps in each location

MAX_LEAK = 3        # Max leak percent

df_data = pd.DataFrame()    # Dataframe of model outputs with noise and leak
df_op = pd.DataFrame()      # Dataframe of leak percentages for all readings
df_ip = pd.DataFrame()      # Dataframe of initial readings without any noise

ll = []
lp = [0]*iter

location = [ "COND"] #, "CO2", "COND"  # Leak Locations CO2 = unit2, COND = unit1


for i in range(iter):
  ll = ll + [random.choice(location)]
  # For no leak, comment out the next line 
  lp[i] = random.randint(0,MAX_LEAK*100)/100  # This will choose a random number for leak percentage. e.

print(ll)
print(lp)

for i in range(iter):
  # index = ll.index(ll[i])
  # index = index*8 - 1

  y_df = output_data(lp[i], ll[i], n_timestamps)
  df_op = df_op.append(y_df)

  df_generated,df_mass, df_m = model_pd( lp[i], ll[i], n_timestamps)
  # print(df_m)
  df_data = df_data.append(df_generated)
  df_ip = df_ip.append(df_m)

index = df_op.index
df_ip = df_ip.set_index(index)

### Combining all the dataframes into 1 dataframe
df_final = pd.concat([df_data, df_op], axis=1, join='outer')
df_final = pd.concat([df_ip, df_final], axis=1, join='outer')

['COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND', 'COND']
[1.08, 1.17, 1.67, 1.39, 1.34, 2.23, 2.93, 0.33, 1.05, 1.29, 1.5, 2.16, 2.4, 1.32, 0.98, 2.62, 0.9, 0.88, 2.71, 1.19]


100%|██████████| 20/20 [00:00<00:00, 33.32it/s]


# main

In [ ]:
CO2 = df_final.iloc[:,6]
df_final.iloc[:,6] = df_final.iloc[:,7]
df_final.iloc[:,7] = CO2
df_final = df_final.rename(columns={'CO2': 'N2', 'N2': 'CO2'})


In [ ]:
df_final 

,C1,C2,C3,C4,C5,C6,N2,CO2,FEED_SC_C1,FEED_SC_C2,FEED_SC_C3,FEED_SC_C4,FEED_SC_C5,FEED_SC_C6,FEED_SC_N2,FEED_SC_CO2,COND_C1,COND_C2,COND_C3,COND_C4,COND_C5,COND_C6,COND_N2,COND_CO2,FEED_GSU_C1,FEED_GSU_C2,FEED_GSU_C3,FEED_GSU_C4,FEED_GSU_C5,FEED_GSU_C6,FEED_GSU_N2,FEED_GSU_CO2,CO2_C1,CO2_C2,CO2_C3,CO2_C4,CO2_C5,CO2_C6,CO2_N2,CO2_CO2,...,FEED_SC_C1_y,FEED_SC_C2_y,FEED_SC_C3_y,FEED_SC_C4_y,FEED_SC_C5_y,FEED_SC_C6_y,FEED_SC_N2_y,FEED_SC_CO2_y,COND_C1_y,COND_C2_y,COND_C3_y,COND_C4_y,COND_C5_y,COND_C6_y,COND_N2_y,COND_CO2_y,FEED_GSU_C1_y,FEED_GSU_C2_y,FEED_GSU_C3_y,FEED_GSU_C4_y,FEED_GSU_C5_y,FEED_GSU_C6_y,FEED_GSU_N2_y,FEED_GSU_CO2_y,CO2_C1_y,CO2_C2_y,CO2_C3_y,CO2_C4_y,CO2_C5_y,CO2_C6_y,CO2_N2_y,CO2_CO2_y,FEED_NGL_C1_y,FEED_NGL_C2_y,FEED_NGL_C3_y,FEED_NGL_C4_y,FEED_NGL_C5_y,FEED_NGL_C6_y,FEED_NGL_N2_y,FEED_NGL_CO2_y
0,553648.100343,65756.270600,49163.807241,32396.830803,30936.409318,66513.923203,30025.253812,98126.945880,585060,69487.1,51953.2,34234.9,32691.6,70287.7,31728.8,103694,0,0,0,0,32179.7,69187,0,0,575136,68308.4,51072,33654.2,0,0,31190.6,101935,0,0,0,0,0,0,0,101937,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,555172.051720,65937.268890,49299.133726,32486.005127,31021.563739,66697.006979,30107.900220,98397.046499,586671,69678.3,52096.2,34329.2,32781.6,70481.2,31816.1,103980,0,0,0,0,32267.3,69375.3,0,0,576702,68494.3,51211,33745.8,0,0,31275.5,102213,0,0,0,0,0,0,0,102214,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,553363.096146,65722.420916,49138.498942,32380.153725,30920.484028,66479.683498,30009.797562,98076.432583,584759,69451.3,51926.5,34217.3,32674.8,70251.5,31712.5,103641,0,0,0,0,32163.3,69151.8,0,0,574844,68273.7,51046,33637.1,0,0,31174.7,101884,0,0,0,0,0,0,0,101885,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,554552.485305,65863.683562,49244.116390,32449.751072,30986.943987,66622.573791,30074.300106,98287.236387,586016,69600.6,52038.1,34290.8,32745,70402.5,31780.6,103864,0,0,0,0,32231.7,69298.8,0,0,576065,68418.8,51154.5,33708.6,0,0,31241,102100,0,0,0,0,0,0,0,102101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,555201.503832,65940.766890,49301.749066,32487.728523,31023.209446,66700.545283,30109.497457,98402.266504,586702,69682,52099,34331,32783.4,70484.9,31817.8,103985,0,0,0,0,32269,69379,0,0,576732,68497.9,51213.7,33747.6,0,0,31277.1,102218,0,0,0,0,0,0,0,102220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,1.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,529298.763167,62864.322442,47001.592436,30972.024403,29575.831975,63588.653628,28704.748912,93811.341637,559329,66431,49668.3,32729.3,31253.9,67196.5,30333.4,99133.9,0,0,0,0,30746.1,66104.7,0,0,549514,65265.3,48796.7,32154.9,0,0,29801,97394.2,0,0,0,0,0,0,0,97395.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.19,1.19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,528540.223940,62774.231442,46934.234350,30927.638327,29533.446785,63497.524588,28663.612073,93676.900396,558528,66335.8,49597.1,32682.4,31209.1,67100.2,30289.9,98991.8,0,0,0,0,3070

In [ ]:
N_input=5
N = 1000
SEED = 0 


*   Possible leak locations are units
*   Noises are representing each pipline
*   product{unit number}: what we are seprating from the main pipe in {unit number}
*   output{unit number}: what is remained in the main pipe after {unit number}
*   y (sensor readings) = [input,product{unit number},output{unit number},...]




In [ ]:
#Matrix diagram for the unit's proiducts 
matrix1 = (torch.Tensor([0,0,0,0,1,1,0,0]))  #matrix for unit 1
matrix2 = (torch.Tensor([0,0,0,0,0,0,0,1]))  #matrix for unit 2

GasComponents = len(matrix1)       # Number of components in the gas stream

leak=torch.tensor([0.,0.])        # #leak  = #unit
noise = torch.tensor([0.] * GasComponents)
x = torch.tensor([1.] * GasComponents)

class unit():
  def __init__(self,matrix1=matrix1,matrix2=matrix2,leak=leak,noise=noise):
    self.matrix1=torch.diag(matrix1)
    self.matrix2=torch.diag(matrix2)
    self.leak = leak
    self.noise = torch.add(1,(noise))#.reshape(-1,1)


  def process(self,x):
    #Calculation before noise and leak
    
    x_0=(x*(1-leak[0]))
    product1 = torch.matmul(x_0,torch.transpose(self.matrix1,0,1))
    output1 = (x_0-product1)

    x_1 = (output1*(1-leak[1]))
    product2 = torch.matmul(x_1,torch.transpose(self.matrix2,0,1))
    output2  = (x_1-product2)
    B = torch.cat([x,product1,output1,product2,output2]).reshape(-1,len(x))
    
    #add noise
    B = torch.mul(self.noise,B)

    return B

#Output Example 
#no leak, no noise
unit().process(x)

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 1., 1., 1., 0., 0., 1., 0.]])

In [ ]:
#Convert Data Sets to Tensor
y_dataset = torch.tensor(df_final.iloc[:,8:48].values.astype(np.float32))
x_dataset = torch.tensor(df_final.iloc[:,:8].values.astype(np.float32))

In [ ]:
#Split the datasets
X_train, X_test, y_train, y_test = train_test_split(x_dataset, y_dataset, test_size=0.30, random_state=42)

In [ ]:
class Leakage(torch.nn.Module):
    def __init__(self,inputSize=1, outputSize=1,SEED=0): #default to one-dimensional feature and response
        super().__init__() #run init of torch.nn.Module
        if SEED is not None:
          torch.manual_seed(SEED)
        self.preleakage = (torch.autograd.Variable(torch.tensor([0.,0.]), requires_grad=False))
        self.linear = torch.nn.Linear(inputSize,outputSize)
        self.sigmoid=torch.nn.Sigmoid()
        self.prenoise=torch.nn.Parameter(torch.autograd.Variable(torch.tensor([0.]*len(x)), requires_grad=True))

    def train_noise(self,flag):
        self.prenoise.requires_grad=flag

    def getleakage(self, error):
      #Calculated leak value is stored in here for every step for each components
      self.sigmoid_value = self.sigmoid(self.linear(error))  #error[0:]=error
      return self.sigmoid_value.mean()


    def forward(self, x, y):
        noise = torch.exp(self.prenoise)   
        leak = self.preleakage

        #calculate the sensor readings in every sensor point with noise
        y_calculated = unit(noise = noise,leak = leak).process(x)
        
        #leak_mean calculation = mean((input -(output + product))/ input) 
        #AD: nanmean beacuse we will have some values like 0/0 
        y = y.reshape(-1,len(x))

        leak1_mean = ((abs(y[0] - (y[1] + y[2]))/y[0]).nanmean()).reshape(-1,1) #leak for unit 1
        leak2_mean = ((abs(y[2] - (y[3] + y[4]))/y[2]).nanmean()).reshape(-1,1) #leak for unit 2

        #Call the sigmoid funtion to return the leak location prediction
        self.error = torch.cat([leak1_mean, leak2_mean],0)
        self.getleakage(self.error)


        #Leak is calculated in every step with the formula above 
        self.getleakage(self.error)

        y_calculated = y_calculated.reshape(-1)

        return y_calculated

model=Leakage()
out=model(X_train[0],y_train[0])
calibration_noise_loss=torch.nn.MSELoss()
leakageloss=torch.nn.MSELoss()  
model.error, model.sigmoid_value

(tensor([[0.0088],
         [0.0071]]), tensor([[0.6310],
         [0.6310]], grad_fn=<SigmoidBackward0>))

# Calibrate noise

In [ ]:
training_noise_inputs=X_train
training_noise_outputs=y_train
training_noise_inputs.shape,training_noise_outputs[0].shape

(torch.Size([280, 8]), torch.Size([40]))

In [ ]:
optimizer_noise = torch.optim.Adam(model.parameters())
losses=[]
MAX_iter=training_noise_inputs.shape[0]

for ctr in range(MAX_iter):

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer_noise.zero_grad()

    # get output from the model, given the inputs
    computed_outputs = model(training_noise_inputs[ctr],training_noise_outputs[ctr])

    # get loss for the predicted output
    # print(training_noise_outputs[ctr].shape)
    lossvalue = calibration_noise_loss(training_noise_outputs[ctr], computed_outputs)
    losses.append(lossvalue)

    # get gradients w.r.t to parameters
    lossvalue.backward()
    #print(model.linear.weight.grad.item(),model.linear.bias.grad.item())

    # update parameters
    optimizer_noise.step()
    if ctr%int(MAX_iter/10)==0: #print out data for 10 intermediate steps
      print("iteration {}: loss={:.5f}".format(ctr, lossvalue.item()))

print("final loss={:.5f}".format(lossvalue.item()))

iteration 0: loss=20347023360.00000
iteration 28: loss=21327605760.00000
iteration 56: loss=18396037120.00000
iteration 84: loss=18996490240.00000
iteration 112: loss=16692417536.00000
iteration 140: loss=15755515904.00000
iteration 168: loss=15044265984.00000
iteration 196: loss=14465155072.00000
iteration 224: loss=13664358400.00000
iteration 252: loss=13015195648.00000
final loss=13682293760.00000


# Compute leakage

In [ ]:
leakage_inputs=X_test
leakage_outputs=y_test
LASSO=3.0

In [ ]:
model.train_noise(False)

In [ ]:
optimizer_leakage = torch.optim.Adam(model.parameters())
losses=[]
MAX_iter=leakage_inputs.shape[0]
sigmoid_value_list = []
error_list = []
for ctr in range(MAX_iter):

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer_leakage.zero_grad()

    # get output from the model, given the inputs
    computed_outputs = model(leakage_inputs[ctr],leakage_outputs[ctr])
    error_list.append(model.error.tolist())

    # get loss for the predicted output
    lossvalue = leakageloss(leakage_outputs[ctr],computed_outputs)+LASSO*torch.sum(model.getleakage(model.error)) #leak_calculated_list[0] ##Λ ̃η∗(ℓ1,ℓ2) +C{|ℓ1|+|ℓ2|}
    losses.append(lossvalue)
    sigmoid_value_list.append(model.sigmoid_value.tolist())

    # get gradients w.r.t to parameters
    lossvalue.backward()
    #print(model.linear.weight.grad.item(),model.linear.bias.grad.item())

    # update parameters
    optimizer_leakage.step()
    if ctr%int(MAX_iter/10)==0: #print out data for 10 intermediate steps
      print("iteration {}: loss={:.5f}".format(ctr, lossvalue.item()))

print("final loss={:.5f}".format(lossvalue.item()))

iteration 0: loss=12971496448.00000
iteration 12: loss=12308999168.00000
iteration 24: loss=13371521024.00000
iteration 36: loss=12311791616.00000
iteration 48: loss=13381082112.00000
iteration 60: loss=13940886528.00000
iteration 72: loss=13375050752.00000
iteration 84: loss=12204906496.00000
iteration 96: loss=12421123072.00000
iteration 108: loss=12164384768.00000
final loss=13419713536.00000


# **Deciding by mean:**

In [ ]:
sigmoid_value_torch = torch.tensor(sigmoid_value_list)
sigmoid_ave_value_unit1 = sigmoid_value_torch[:,0].mean()
sigmoid_ave_value_unit2 = sigmoid_value_torch[:,1].mean()

if sigmoid_ave_value_unit1 == sigmoid_ave_value_unit2:
  print("Leak does not exist!")
elif sigmoid_ave_value_unit1 > sigmoid_ave_value_unit2:
  print("Leak is in Unit 2!")
else:
  print("Leak is in Unit 1!")

Leak is in Unit 1!


# **Deciding by voting:**

In [ ]:
leak_location_vote_torch = torch.tensor([0]*len(sigmoid_value_torch))
for test_row in range(len(leak_location_vote_torch)):
  if sigmoid_value_torch[test_row][0]<sigmoid_value_torch[test_row][1]:
    leak_location_vote_torch[test_row] = 1
  elif sigmoid_value_torch[test_row][0]>sigmoid_value_torch[test_row][1]:
    leak_location_vote_torch[test_row] = 2
  else:
    leak_location_vote_torch[test_row] = 0

if int(torch.mode(leak_location_vote_torch,0).values) == 0:
  print("Leak does not exist!")
else:
  print("Leak is in Unit {}!".format(int(torch.mode(leak_location_vote_torch,0).values)))

#leak_location_vote_torch.unique(return_counts=True)
print(location)

Leak is in Unit 1!
['COND']
